In [1]:
!pip install fastapi uvicorn pyngrok pydantic torch torchtext spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 2.3 MB/s eta 0:00:00


In [2]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 17.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install torchtext==0.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.18.0
    Uninstalling torchtext-0.18.0:
      Successfully uninstalled torchtext-0.18.0


In [15]:
import torch
from torchtext import data

# Define the field for text and labels
TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  include_lengths = True)

LABEL = data.LabelField(dtype = torch.float)

# Load the IMDB dataset
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

import random
train_data, valid_data = train_data.split(random_state = random.seed(1234))

# Build the vocabulary using the training data
MAX_VOCAB_SIZE = 25_000
TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE, vectors="glove.6B.100d", unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:19<00:00, 4.37MB/s]
.vector_cache/glove.6B.zip: 862MB [02:39, 5.40MB/s]                           
100%|█████████▉| 399999/400000 [00:28<00:00, 13992.13it/s]


In [16]:
# Get the vocabulary (list of words) from the TEXT field
vocab = TEXT.vocab.stoi

# Specify the directory to save the vocab.txt file
import os
directory = '/content/drive/MyDrive/mentoring/Kelas-NLP/models/'

# Create the directory if it doesn't exist
os.makedirs(directory, exist_ok=True)

# Save vocabulary to a text file
vocab_file_path = os.path.join(directory, 'vocab.txt')

with open(vocab_file_path, 'w') as vocab_file:
    for word, index in vocab.items():
        vocab_file.write(f"{word}\t{index}\n")

print(f"Vocabulary saved to {vocab_file_path}")

Vocabulary saved to /content/drive/MyDrive/mentoring/Kelas-NLP/models/vocab.txt


In [5]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import torch
from torch.nn import functional as F
from torchtext import data
import spacy
from pyngrok import ngrok
import uvicorn
import threading
import os

In [6]:
# Initialize FastAPI
app = FastAPI(title="Sentiment Analysis API", version="1.0")

# Load the tokenizer
nlp = spacy.load('en_core_web_sm')

# Model and Vocabulary Loading
class SentimentRNN(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = torch.nn.LSTM(
            embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout
        )
        self.fc = torch.nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to("cpu"))
        packed_output, (hidden, _) = self.rnn(packed_embedded)
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        return self.fc(hidden)

In [7]:
!ls "/content/drive/My Drive/mentoring/Kelas-NLP/models"

sentiment-model.pt  vocab.txt


In [10]:
# Define Request Model
class SentimentRequest(BaseModel):
    sentence: str

# Load vocabulary from vocab.txt
def load_vocab(vocab_file_path):
    vocab = {}
    with open(vocab_file_path, 'r') as f:
        for line in f:
            line = line.strip()
            # Skip empty lines or lines that don't contain a tab
            if not line or '\t' not in line:
                continue
            word, index = line.split('\t')
            vocab[word] = int(index)
    return vocab

# Load model
def load_model():
    INPUT_DIM = 25002  # Change based on your training setup
    EMBEDDING_DIM = 100
    HIDDEN_DIM = 256
    OUTPUT_DIM = 1
    N_LAYERS = 2
    BIDIRECTIONAL = True
    DROPOUT = 0.5
    PAD_IDX = 1  # Replace with `TEXT.vocab.stoi[TEXT.pad_token]` used during training

    # Path to the model in Google Drive
    drive_path = '/content/drive/My Drive/mentoring/Kelas-NLP/models/sentiment-model.pt'

    # Ensure model file exists
    if not os.path.exists(drive_path):
        raise FileNotFoundError(f"Model file not found at {drive_path}")

    # Load model
    model = SentimentRNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)
    model.load_state_dict(torch.load(drive_path, map_location=torch.device("cpu")))
    model.eval()
    return model

# Load vocab from vocab.txt
vocab_file_path = '/content/drive/MyDrive/mentoring/Kelas-NLP/models/vocab.txt'
vocab = load_vocab(vocab_file_path)

# Initialize model
model = load_model()

# Define prediction function
def predict_sentiment(sentence: str):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    # Use the loaded vocabulary to get indices of words
    indexed = [vocab.get(t, vocab.get('<unk>', 0)) for t in tokenized]  # Handle unknown words
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

<ipython-input-10-7dbb51800dbf>:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(drive_path, map_location=torch.device("cpu")))


In [11]:
# API Routes
@app.get("/")
def read_root():
    return {"message": "Welcome to the Sentiment Analysis API"}

@app.post("/predict/")
def analyze_sentiment(request: SentimentRequest):
    try:
        sentiment_score = predict_sentiment(request.sentence)
        sentiment = "positive" if sentiment_score >= 0.5 else "negative"
        return {"sentence": request.sentence, "sentiment": sentiment, "score": sentiment_score}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error occurred during prediction: {e}")

@app.post("/test/")
def test_analyze_sentiment(request: SentimentRequest):
    try:
        return {"sentence": request.sentence}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error occurred during prediction: {e}")

In [12]:
!ngrok config add-authtoken YOUR_NGROK_AUTHTOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
# Function to run FastAPI server
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start ngrok and FastAPI server
ngrok_tunnel = ngrok.connect(8000)
print(f"Public URL: {ngrok_tunnel.public_url}")

# Run FastAPI in a thread
thread = threading.Thread(target=run_app)
thread.start()

Public URL: https://dbdf-34-45-180-94.ngrok-free.app


In [14]:
import requests

url = "https://dbdf-34-45-180-94.ngrok-free.app/predict"
data = {"sentence": "This film is great"}
response = requests.post(url, json=data)
print(response.json())

INFO:     34.45.180.94:0 - "POST /predict HTTP/1.1" 307 Temporary Redirect
INFO:     34.45.180.94:0 - "POST /predict/ HTTP/1.1" 200 OK
{'sentence': 'This film is great', 'sentiment': 'positive', 'score': 0.9459415674209595}
